**Install Python Speech Features** - This is python library to handle speech data.

In [1]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5889 sha256=a67f3eaa96408354c6a58eb81125ba985de577097a8862c7bc83718cff119b02
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
import os

Download Dataset Directly to Colab

In [3]:
!wget "https://zenodo.org/record/1188976/files/Audio_Song_Actors_01-24.zip"

--2019-12-04 18:14:23--  https://zenodo.org/record/1188976/files/Audio_Song_Actors_01-24.zip
Resolving zenodo.org (zenodo.org)... 188.184.95.95
Connecting to zenodo.org (zenodo.org)|188.184.95.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225505317 (215M) [application/octet-stream]
Saving to: ‘Audio_Song_Actors_01-24.zip’

Audio_Song_Actors_0 100%[===================>] 215.06M  9.57MB/s    in 33s     

2019-12-04 18:14:58 (6.42 MB/s) - ‘Audio_Song_Actors_01-24.zip’ saved [225505317/225505317]



In [0]:
os.mkdir("speech")

Extraction of speech data in speech Folder

In [5]:
!unzip \*.zip -d "speech"

Archive:  Audio_Song_Actors_01-24.zip
   creating: speech/Actor_01/
  inflating: speech/Actor_01/03-02-01-01-01-01-01.wav  
  inflating: speech/Actor_01/03-02-01-01-01-02-01.wav  
  inflating: speech/Actor_01/03-02-01-01-02-01-01.wav  
  inflating: speech/Actor_01/03-02-01-01-02-02-01.wav  
  inflating: speech/Actor_01/03-02-02-01-01-01-01.wav  
  inflating: speech/Actor_01/03-02-02-01-01-02-01.wav  
  inflating: speech/Actor_01/03-02-02-01-02-01-01.wav  
  inflating: speech/Actor_01/03-02-02-01-02-02-01.wav  
  inflating: speech/Actor_01/03-02-02-02-01-01-01.wav  
  inflating: speech/Actor_01/03-02-02-02-01-02-01.wav  
  inflating: speech/Actor_01/03-02-02-02-02-01-01.wav  
  inflating: speech/Actor_01/03-02-02-02-02-02-01.wav  
  inflating: speech/Actor_01/03-02-03-01-01-01-01.wav  
  inflating: speech/Actor_01/03-02-03-01-01-02-01.wav  
  inflating: speech/Actor_01/03-02-03-01-02-01-01.wav  
  inflating: speech/Actor_01/03-02-03-01-02-02-01.wav  
  inflating: speech/Actor_01/03-02-0

Import all Necessary libraries

In [0]:
from sklearn.model_selection import train_test_split

# tensorflow imports
import tensorflow as tf
import keras

# tf.keras imports
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

from tensorflow.keras.layers import Dense, Conv1D, Activation, Dropout
from tensorflow.keras.layers import BatchNormalization, Input, MaxPooling1D

# import for showing the confusion matrix

import numpy as np

**MFCC** features are extracted for each .wav files are extracted

In [0]:
from python_speech_features import mfcc
import os
import librosa

def wav_to_mfcc(wav_filename, num_cepstrum):
    """ extract MFCC features from a wav file
    
    :param wav_filename: filename with .wav format
    :param num_cepstrum: number of cepstrum to return
    :return: MFCC features for wav file
    """
    sig, rate = librosa.load(wav_filename)
    
    mfcc_features = librosa.feature.mfcc(sig, rate, n_mfcc=num_cepstrum)
    #mfcc_features.astype('float32')
    return mfcc_features

Using PowerTrasformer from sklearn Library these MFCC features are normalized and Standardized for better performance.

In [0]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer()

In [0]:
data_dir = "/content/speech"

Data is Labeled and and Loaded in matrix and matrix is padded with 0 for making all of them 0.

In [0]:
def load_data(data_dir):
  data = []
  labels = []

  for folder in os.listdir(data_dir):
    path_ = os.path.join(data_dir,folder)
    for i in os.listdir(path_):
      path_final = os.path.join(path_,i)
      
      mfcc = wav_to_mfcc(path_final, 40)
           
      str = i[6:8]
      if(str == "01"):
        labels.append(1)
      elif(str == "02"):
        labels.append(2)
      elif(str == "03"):
        labels.append(3)
      elif(str == "04"):
        labels.append(4)
      elif(str == "05"):
        labels.append(5)
      elif(str == "06"):
        labels.append(6)
      elif(str == "07"):
        labels.append(7)
      elif(str == "08"):
        labels.append(8)

      labels = labels


      data2 = []
      for i in range(len(mfcc)):
        data3 =[]
        for j in range(len(mfcc[0])):
          data3.append(mfcc[i][j])
        data3 = np.pad(data3, (0, 300-len(data3)))
        
        data2.append(data3)
      data2 = np.asarray(data2)
      data2 = data2.transpose() # transform takes n_samples, n_features but orignally in form n_features, n_samples
      
      pt = PowerTransformer()
      pt.fit(data2)
      data2 = pt.transform(data2)

      data.append(data2)
  
  data = np. asarray(data)    

  labels = tf.keras.utils.to_categorical(labels)
  return data,labels

In [0]:
data_dir = "/content/speech"
data,labels = load_data(data_dir)

Checking the shape of Data.

In [15]:
data.shape

(1012, 300, 40)

Dividing Data in TRAIN, TEST and SPLIT

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size = 0.05, shuffle=True)

In [17]:
Y_train.shape

(961, 7)

In [19]:
X_train.shape

(961, 300, 40)

Creating the model using Convolutional 1D Layer, Maxpooling 1D Layer, Dropout Layer, BatchNormaliaztion Layer and Dense Layer

In [0]:
def create_model():
  model = Sequential()
  
  conv1 = model.add(Conv1D(16,kernel_size=3, padding='same', activation='relu', input_shape=(300,40)))
  max1 = model.add(MaxPooling1D(pool_size=2))

  conv3 = model.add(Conv1D(32,kernel_size=3, padding='same', activation='relu'))
  max2 = model.add(MaxPooling1D(pool_size=2))
  
  drop1 = model.add(Dropout(0.5))
  conv5 = model.add(Conv1D(64,kernel_size=3, padding='same', activation='relu'))
  max3 = model.add(MaxPooling1D(pool_size=2))
  
  bn = model.add(BatchNormalization())
  fl = model.add(Flatten())
  drop2 = model.add(Dropout(0.5))
  dense1 = model.add(Dense(128, activation='relu', kernel_regularizer = regularizers.l2(0.01)))
  
  dense2 = model.add(Dense(7, activation='softmax'))
  
  
  model.compile(optimizer = 'adam', loss = tf.keras.losses.categorical_crossentropy, metrics = ["accuracy"])
   
  model.summary()
  
  return model

We here Initialize the Model and Run it

In [26]:
model = create_model()
model_hist =   model.fit(X_train, Y_train, epochs = 150, batch_size = 16, validation_split=0.05) 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 300, 16)           1936      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 150, 16)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 150, 32)           1568      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 75, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 75, 64)            6208      
_______________________________________

Evaluating the model on Test data

In [27]:
model.evaluate(X_test, Y_test, verbose=2)

51/51 - 0s - loss: 0.6567 - acc: 0.8824


[0.6566636866214228, 0.88235295]